In [47]:
class Matrix:
    def __init__(self, matrix):
        self.matrix = matrix #инициализация объекта матрицы, принимает двумерный список и сохраняет его

    def __str__(self):
        return '\n'.join([' '.join(map(str, row)) for row in self.matrix]) #представление объекта в виде строки. преобразует каждую строку матрицы в строку символов и объединяет их в одну строку с разделителями больше текста больше текста 

    def _elementwise_operation(self, other, operator):  #метод для выполениения поэлементных операций +-*/ матрицы на число/другую матрицу #other - число или другая матрица, operator - функция для выполнения операции
        if isinstance(other, (int, float)): #если other - число, то выполняем операцию с числом над каждым элементом матрицы
            result = [[operator(x, other) for x in row] for row in self.matrix]
        elif isinstance(other, Matrix):#если other - объект класса Matrix, то выполняем операцию с соответствующими элементами двух матриц. для этого используем функцию zip, которая объединяет соответствующие элементы двух рядов матриц. лямбда (далее) -функция оператор и применяется к этим парам элементов.
            result = [[operator(x, y) for x, y in zip(row1, row2)] for row1, row2 in zip(self.matrix, other.matrix)]
        return Matrix(result) # возвращение новой матрицы с результатами операции 

    def __add__(self, other): # add перегружает оператор + для объектов класса Matrix. Он вызывает _elementwise_operation с оператором сложения для выполнения поэлементного сложения матриц.
        return self._elementwise_operation(other, lambda x, y: x + y)

    def __sub__(self, other): # sub перегружает оператор - для объектов класса Matrix. Он вызывает _elementwise_operation с оператором вычитания для выполнения поэлементного вычитания матриц.
        return self._elementwise_operation(other, lambda x, y: x - y)

    def __truediv__(self, other): # truediv перегружает оператор / для объектов класса Matrix. Он вызывает _elementwise_operation с оператором деления для выполнения поэлементного деления матриц, если делитель равен нулю, возвращается 0.
        return self._elementwise_operation(other, lambda x, y: x / y if y != 0 else 0)

    def __mul__(self, other): # mul перегружает оператор * для объектов класса Matrix. Он вызывает _elementwise_operation с оператором умножения для выполнения поэлементного умножения матриц.
        return self._elementwise_operation(other, lambda x, y: x * y)

    def __matmul__(self, other): # matmul перегружает оператор @ для объектов класса Matrix. Он выполняет умножение матриц по правилам матричного умножения. Для этого используется генератор списков, где каждый элемент новой матрицы вычисляется как сумма произведений элементов строк первой матрицы и столбцов второй матрицы. Здесь используется zip для удобного объединения элементов строк первой матрицы и столбцов второй матрицы.
        result = [[sum(a * b for a, b in zip(row1, col2)) for col2 in zip(*other.matrix)] for row1 in self.matrix]
        return Matrix(result)

    def shape(self):# shape возвращает кортеж из двух значений: количество строк и количество столбцов матрицы.
        return (len(self.matrix), len(self.matrix[0]))

    def T(self): # T выполняет транспонирование матрицы, меняя строки и столбцы местами. для этого используется zip(*self.matrix), что позволяет получить кортеж столбцов матрицы. затем создается новая матрица из этих столбцов.
        result = [[x for x in row] for row in zip(*self.matrix)]
        return Matrix(result)

#zip позволяет объединить элементы матрицы для выполнения операций над соответствующими парами элементов, а lambda определяет функцию, применяемую к этим парам элементов.


In [48]:
%pip install -q ipytest pytest

import numpy as np
import ipytest
import pytest
ipytest.autoconfig()

Note: you may need to restart the kernel to use updated packages.


In [49]:
%%ipytest

matrix1 = Matrix([[10,11,12],[13,14,15],[16,17,18]])
matrix2 = Matrix([[10,11,12],[13,14,15],[16,17,18]])

@pytest.mark.parametrize("matrix1,matrix2", [[matrix1, matrix2]])
def test_matrix_add_matrix(matrix1: Matrix, matrix2: Matrix):
    assert (
        (np.array((matrix1 + matrix2).matrix) == (np.array(matrix1.matrix) + np.array(matrix2.matrix))).all()
    ), 'add section with matrices is not working right'

@pytest.mark.parametrize("matrix1,number", [[matrix1, 13.2]])
def test_matrix_add_number(matrix1: Matrix, number: int | float):
    assert (
        (np.array((matrix1 + number).matrix) == np.array(matrix1.matrix) + number).all()
    ), 'add section with matrix and number is not working right'

@pytest.mark.parametrize("matrix1,matrix2", [[matrix1, matrix2]])
def test_matrix_sub_matrix(matrix1: Matrix, matrix2: Matrix):
    assert (
        (np.array((matrix1 - matrix2).matrix) == np.array(matrix1.matrix) - np.array(matrix2.matrix)).all()
    ), 'sub section with matrices is not working right'

@pytest.mark.parametrize("matrix1,number", [[matrix1, 12.2]])    
def test_matrix_sub_number(matrix1: Matrix, number: int | float):
    assert (
        (np.array((matrix1 - number).matrix) == np.array(matrix1.matrix) - number).all()
    ), 'sub section with matrix and number is not working right'

@pytest.mark.parametrize("matrix1", [matrix1])    
def test_matrix_transpose(matrix1: Matrix):
    assert (
        (np.array(matrix1.T().matrix) == np.array(matrix1.matrix).T).all()
    ), 'transpose section is not working right'
    
# -----------------------------------------

@pytest.mark.parametrize("matrix1,matrix2", [[matrix1, matrix2]])
def test_matrix_mul_matrix(matrix1: Matrix, matrix2: Matrix):
    assert (
        (np.array((matrix1 * matrix2).matrix) == np.array(matrix1.matrix) * np.array(matrix2.matrix)).all()
    ), 'mul section with matrices is not working right'

@pytest.mark.parametrize("matrix1,number", [[matrix1, 13]])
def test_matrix_mul_number(matrix1: Matrix, number: int | float):
    assert (
        (np.array((matrix1 * number).matrix) == np.array(matrix1.matrix) * number).all()
    ), 'mul section with matrix and number is not working right'

@pytest.mark.parametrize("matrix1,matrix2", [[matrix1, matrix2]])    
def test_matrix_div_matrix(matrix1: Matrix, matrix2: Matrix):
    assert (
        (np.array((matrix1 / matrix2).matrix) == np.array(matrix1.matrix) / np.array(matrix2.matrix)).all()
    ), 'div section with matrices is not working right'

@pytest.mark.parametrize("matrix1,number", [[matrix1, 2]])    
def test_matrix_div_number(matrix1: Matrix, number: int | float):
    assert (
        (np.array((matrix1 / number).matrix) == np.array(matrix1.matrix) / number).all()
    ), 'div section with matrix and number is not working right'

@pytest.mark.parametrize("matrix1,matrix2", [[matrix1, matrix2.T()]])    
def test_matrix_matmul_matrix(matrix1: Matrix, matrix2: Matrix):
    assert(
        (np.array((matrix1 @ matrix2).matrix) == np.array(matrix1.matrix) @ np.array(matrix2.matrix)).all()
    ), 'matmul section is not working right, and don\'t forget about T() if needed'

..........                                                                                   [100%]
10 passed in 0.08s
